In [1]:
from utils import get_webpage_content, extract_params, process_model_output, filter_json
from json5 import loads
import pandas as pd
pd.set_option('display.max_colwidth', None)
from tqdm import tqdm

In [2]:
example_single = "Find me a beer brewed before 08-2024 with ABV more than 200." 
processed_text = extract_params(example_single)
# print('*' + processed_text)
processed_text = process_model_output(processed_text)
# print('**' + processed_text)
processed_text = loads(processed_text)
print(str(processed_text))
print('-----------------')
processed_text = filter_json(processed_text)
print(str(processed_text))

{'abv_gt': 200, 'brewed_before': '08-2024'}
-----------------
Error: Invalid number format, abv_gt is above 100
Error: Invalid date format, brewed_before is not in mm-yyyy format Q



In [ ]:
with open('test_cases.txt', 'r') as f:
    examples = f.read().split('\n')

examples_processed = [example.split("\"")[1].split("\"")[0] for example in examples]
examples_cases = [example.split("(")[1].split(")")[0] if len(example.split("(")) > 1 else None for example in examples]

In [ ]:
examples_list = []
case_list = []
output_list = []

for i in tqdm(range(100)):
    examples_list.append(examples[i])
    case_list.append(examples_cases[i])
    count = 0
    while count < 2:
        try:
            output_list.append(filter_json(loads(process_model_output(extract_params(examples_processed[i])))))
            count = 2
        except:
            count += 1
            if count == 2:
                output_list.append("{}")
            continue

In [ ]:
pd.DataFrame({"Example": examples_list, "Edge_Case": case_list, "Output": output_list}).to_csv("test_results_v4.csv", index=True)

In [ ]:
for sample in output_list:
    try:
        json_input = loads(sample)
        json_input = {key: json_input[key] for key in json_input.keys() if key in ["abv_gt", "abv_lt", "ibu_gt", "ibu_lt", "ebc_gt", "ebc_lt", "brewed_before", "brewed_after"]}
        json_input = {key: json_input[key] for key in json_input.keys() if json_input[key] is not None}
        json_input = {key: json_input[key] for key in json_input.keys() if json_input[key] != ""}
        json_input = {key: json_input[key] for key in json_input.keys() if json_input[key].strip() != ""}
        if "brewed_before" in json_input.keys():
            if len(json_input["brewed_before"].split("-")) != 2:
                json_input.pop("brewed_before")
            elif len(json_input["brewed_before"].split("-")[0]) != 2 or len(json_input["brewed_before"].split("-")[1]) != 4:
                json_input.pop("brewed_before")
        if "brewed_after" in json_input.keys():
            if len(json_input["brewed_after"].split("-")) != 2:
                json_input.pop("brewed_after")
            elif len(json_input["brewed_after"].split("-")[0]) != 2 or len(json_input["brewed_after"].split("-")[1]) != 4:
                json_input.pop("brewed_after")

        # check the mm is below 13 and yyyy is below current date
        if "brewed_before" in json_input.keys():
            if int(json_input["brewed_before"].split("-")[0]) > 12 or int(json_input["brewed_before"].split("-")[1]) > 2023:
                json_input.pop("brewed_before")

        if "brewed_after" in json_input.keys():
            if int(json_input["brewed_after"].split("-")[0]) > 12 or int(json_input["brewed_after"].split("-")[1]) > 2023:
                json_input.pop("brewed_after")
        
        print(json_input)
    except:
        print('error', sample)

In [ ]:
model_output_json

In [ ]:
query_string = "&".join([f"{key}={value}" for key, value in model_output_json.items()])

In [ ]:
query_string

In [ ]:
content = get_webpage_content("https://api.punkapi.com/v2/beers?" + 'hops=hoppy')
content = loads(content)

In [ ]:
len(content)

In [ ]:
content

In [ ]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
tokenizer = openai.